In [493]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import json
import folium

In [494]:
# Path to your csv file
csv_file_path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus/cooling_boiler_trimmed_2.csv'

# Import 
cooling_boiler_df = pd.read_csv(csv_file_path)  

# Verify the import
print(cooling_boiler_df.dtypes.to_string())

/var/folders/6r/kznwmt6j1rsfwtfplchqpwj00000gn/T/ipykernel_25196/2028054111.py:5: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  cooling_boiler_df = pd.read_csv(csv_file_path)


utility_id                                              int64
state                                                  object
plant_code                                              int64
plant_name                                             object
year                                                    int64
month                                                   int64
generator_id                                           object
boiler_id                                              object
cooling_id                                             object
generator_primary_technology                           object
fuel_consumption_from_all_fuel_types_mmbtu            float64
water_withdrawal_volume_million_gallons               float64
water_consumption_volume_million_gallons              float64
cooling_unit_hours_in_service                         float64
avg_distance_water_intake                             float64
cooling_type_1_860                                     object
cooling_

In [495]:
!pip install geopandas

# Steps 4 - 6: Find GeoJSON; Data Wrangle; Data Clean

In [496]:
import geopandas as gpd

# Load the GeoJSON file
geo_df = gpd.read_file('/Users/amyzhang/Desktop/A6_Dashboard/6.3/Power_Plants.geojson')

# Quick peek
geo_df.head()


,OBJECTID,Plant_Code,Plant_Name,Utility_ID,Utility_Name,sector_name,Street_Address,City,County,State,...,Nuclear_MW,Crude_MW,Solar_MW,Wind_MW,Other_MW,Source,Period,Longitude,Latitude,geometry
0,35853,1,Sand Point,63560,"Sand Point Generating, LLC",Electric Utility,100 Power Plant Way,Sand Point,Aleutians East,Alaska,...,NaN,1.3,NaN,0.4,NaN,"EIA-860, EIA-860M and EIA-923",202406,-160.497222,55.339722,POINT (-160.49722 55.33972)
1,35854,2,Bankhead Dam,195,Alabama Power Co,Electric Utility,19001 Lock 17 Road,Northport,Tuscaloosa,Alabama,...,NaN,NaN,NaN,NaN,NaN,"EIA-860, EIA-860M and EIA-923",202406,-87.356823,33.458665,POINT (-87.35682 33.45867)
2,35855,3,Barry,195,Alabama Power Co,Electric Utility,North Highway 43,Bucks,Mobile,Alabama,...,NaN,NaN,NaN,NaN,NaN,"EIA-860, EIA-860M and EIA-923",202406,-88.010300,31.006900,POINT (-88.0103 31.0069)
3,35856,4,Walter Bouldin Dam,195,Alabama Power Co,Electric Utility,750 Bouldin Dam Road,Wetumpka,Elmore,Alabama,...,NaN,NaN,NaN,NaN,NaN,"EIA-860, EIA-860M and EIA-923",202406,-86.283056,32.583889,POINT (-86.28306 32.58389)
4,35857,9,Copper,5701,El Paso Electric Co,Electric Utility,651 Hawkins Blvd.,El Paso,El Paso,Texas,...,NaN,NaN,NaN,NaN,NaN,"EIA-860, EIA-860M and EIA-923",202406,-106.375000,31.756900,POINT (-106.375 31.7569)


In [497]:
geo_df.columns

Index(['OBJECTID', 'Plant_Code', 'Plant_Name', 'Utility_ID', 'Utility_Name',
       'sector_name', 'Street_Address', 'City', 'County', 'State', 'Zip',
       'PrimSource', 'source_desc', 'tech_desc', 'Install_MW', 'Total_MW',
       'Bat_MW', 'Bio_MW', 'Coal_MW', 'Geo_MW', 'Hydro_MW', 'HydroPS_MW',
       'NG_MW', 'Nuclear_MW', 'Crude_MW', 'Solar_MW', 'Wind_MW', 'Other_MW',
       'Source', 'Period', 'Longitude', 'Latitude', 'geometry'],
      dtype='object')

In [498]:
cooling_boiler_df.shape

(751500, 71)

In [499]:
geo_df.shape

(12798, 33)

### Merge the two Dataframes

In [501]:
# First, standardize column names
geo_df = geo_df.rename(columns={'Plant_Code': 'plant_code'})

# Merge on 'plant_code'
merged_df = cooling_boiler_df.merge(geo_df[['plant_code', 'Plant_Name', 'State', 'Latitude', 'Longitude', 'geometry']],
                                     on='plant_code',
                                     how='left')

# Sanity check
print(merged_df.shape)
merged_df[['plant_code', 'Plant_Name', 'Latitude', 'Longitude']].head()


(751500, 76)


,plant_code,Plant_Name,Latitude,Longitude
0,3,Barry,31.0069,-88.0103
1,3,Barry,31.0069,-88.0103
2,3,Barry,31.0069,-88.0103
3,3,Barry,31.0069,-88.0103
4,3,Barry,31.0069,-88.0103


### Next sanity check: How many plants are missing location info?

In [503]:
missing_geo = merged_df['Latitude'].isna().sum()
print(f"Plants missing spatial data: {missing_geo}")

Plants missing spatial data: 73668


In [504]:
missing_plants = merged_df[merged_df['Latitude'].isna()]['plant_code'].nunique()
total_plants = merged_df['plant_code'].nunique()

print(f"Unique plants missing coordinates: {missing_plants} out of {total_plants}")

Unique plants missing coordinates: 211 out of 1009


#### Comments from ChatGPT: 
"That’s actually a *very respectable match rate*—**~79%** plant-level coverage—and totally workable for mapping 🎯

**Here’s what this means:**
- You have spatial data for **798** of **1009** unique plants
- The unmatched 211 might be older, inactive, renamed, or miskeyed plants
- For visual analysis and communication, 79% coverage is solid"

### Investigate unmatched records

In [507]:
# Filter for plants with missing lat/lon (i.e., no geo match)
unmatched_plants = merged_df[merged_df['Latitude'].isna()].copy()

# Show unique plants
unmatched_summary = unmatched_plants[['plant_code', 'plant_name', 'water_source_name']].drop_duplicates()

print(f"Unique unmatched plants: {unmatched_summary.shape[0]}")
unmatched_summary.head(20)


Unique unmatched plants: 241


,plant_code,plant_name,water_source_name
228,7,Gadsden,Coosa River
252,8,Gorgas,Warrior River
540,50,Widows Creek,Tennessee River
636,51,Dolet Hills,Toledo Bend Lake
768,87,Escalante,Wells
1500,167,Cecil Lynch,Arkansas River
1524,168,Hamilton Moses,L'Anquille River
1548,169,Harvey Couch,Wells
1620,173,Robert E Ritchie,Mississippi River
1680,207,St Johns River Power Park,St. Johns River


In [508]:
unmatched_codes = unmatched_summary['plant_code'].unique()
missing_from_geo = geo_df[~geo_df['plant_code'].isin(unmatched_codes)]

print(f"Are any of the 'missing' plant codes actually in the GeoJSON under a different format? {missing_from_geo.shape[0]} potential matches.")


Are any of the 'missing' plant codes actually in the GeoJSON under a different format? 12798 potential matches.


In [509]:
# Export data to csv
path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus'
output_file = 'geo_unmatched_plants.csv'
full_path = os.path.join(path, output_file)

# Save the dataframe
unmatched_plants.to_csv(full_path, index=False)

# Check if the file was created successfully
if os.path.exists(full_path):
    print(f"Saving confirmed: '{output_file}' has been created successfully.")
else:
    print("Error: File was not saved.")

Saving confirmed: 'geo_unmatched_plants.csv' has been created successfully.


### 🔧 Geographic Data Filtering Decision

The EIA GeoJSON file used for this analysis includes latitude and longitude data only for power plants that remain operational as of 2024. Upon reviewing the unmatched entries from our full dataset, we found that many correspond to facilities that have been retired or decommissioned in recent years (e.g., Gadsden, AL — retired 2023; Gorgas, AL — retired 2019) (source: Global Energy Monitor - Wiki)

Since the goal of our predictive model is to support future oversight and accountability efforts, we have limited our analysis to currently operating plants. This ensures alignment with available spatial data and focuses our insights on infrastructure still relevant to ongoing environmental and regulatory concerns. Retired plants have been excluded from geospatial visualizations and related analyses.


### Filter for matched plants and prep for mapping

In [512]:
mapped_df = merged_df[merged_df['Latitude'].notna()].copy()

In [513]:
mapped_df.shape

(677832, 76)

In [514]:
mapped_df['generator_fuel_type'].value_counts(dropna=False)

generator_fuel_type
(NG) Natural Gas                                                   490416
(BIT) Bituminous Coal                                               71856
(SUB) Subbituminous Coal                                            42144
(BLQ) Black Liquor                                                  20172
(NUC) Nuclear                                                       13728
(BFG) Blast Furnace Gas                                              8748
(RC) Refined Coal                                                    8328
(RFO) Residual Fuel Oil                                              6456
(OG) Other Gas                                                       3564
(WDS) Wood/Wood Waste Solids                                         2076
(DFO) Distillate Fuel Oil                                            1752
(SUN) Solar                                                          1620
(LIG) Lignite Coal                                                   1452
(AB) Agricultural 

# 🗺️ 7a) MAP no.1: Anomaly-focused modeling

### Cleaning & Outlier Handling

To address potential data quality issues, we examined `water_ratio`, a derived field representing the proportion of water consumed versus withdrawn for cooling. In thermoelectric systems, this ratio should logically fall between 0 and 1. However, our data includes a subset of records with:
- Ratios greater than 1
- Ratios less than 0
- Exact zero values

Rather than discarding these anomalies, we preserved them and created a `suspicious_ratio_flag` to identify records that may reflect reporting inconsistencies, data entry errors, or atypical operational behavior.

This approach allows us to:
- Preserve potential signals for predictive modeling
- Meet the data wrangling requirements of the assignment
- Explore possible patterns in anomalous reporting practices


In [517]:
cooling_boiler_df['suspicious_ratio_flag'] = (
    cooling_boiler_df['ratio_gt_1'] | 
    cooling_boiler_df['ratio_lt_0'] | 
    cooling_boiler_df['ratio_eq_0']
)

In [518]:
cooling_boiler_df['suspicious_ratio_flag'].value_counts()


suspicious_ratio_flag
False    666630
True      84870
Name: count, dtype: int64

In [519]:
# Export data to csv
path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus'
output_file = 'cooling_boiler_trimmed_3.csv'
full_path = os.path.join(path, output_file)

# Save the dataframe
cooling_boiler_df.to_csv(full_path, index=False)

# Check if the file was created successfully
if os.path.exists(full_path):
    print(f"Saving confirmed: '{output_file}' has been created successfully.")
else:
    print("Error: File was not saved.")

Saving confirmed: 'cooling_boiler_trimmed_3.csv' has been created successfully.


### Re-making the mapping-ready df

In [521]:
# First, standardize column names
geo_df = geo_df.rename(columns={'Plant_Code': 'plant_code'})

# Merge on 'plant_code'
merged_df = cooling_boiler_df.merge(geo_df[['plant_code', 'Plant_Name', 'State', 'Latitude', 'Longitude', 'geometry']],
                                     on='plant_code',
                                     how='left')

# Sanity check
print(merged_df.shape)


(751500, 77)


In [522]:
mapped_df = merged_df[merged_df['Latitude'].notna()].copy()

In [523]:
mapped_df['suspicious_ratio_flag'].value_counts(dropna=False)

suspicious_ratio_flag
False    597665
True      80167
Name: count, dtype: int64

In [524]:
# Export data to csv
path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus'
output_file = 'cooling_boiler_GEO_trimmed.csv'
full_path = os.path.join(path, output_file)

# Save the dataframe
mapped_df.to_csv(full_path, index=False)

# Check if the file was created successfully
if os.path.exists(full_path):
    print(f"Saving confirmed: '{output_file}' has been created successfully.")
else:
    print("Error: File was not saved.")

Saving confirmed: 'cooling_boiler_GEO_trimmed.csv' has been created successfully.


### 🔧 1. Aggregate Data by plant_code
Make a plant summary (aggregated by plant_code)-- this won't have all the information from mapped_df; so then, will have to re-merge with geo_df.

In [526]:
# Group to plant level
plant_summary = mapped_df.groupby('plant_code').agg({
    'water_ratio': 'mean',
    'suspicious_ratio_flag': 'sum'  # Count of suspicious records per plant
}).reset_index()

plant_summary.rename(columns={
    'suspicious_ratio_flag': 'suspicious_count',
    'water_ratio': 'avg_water_ratio'
}, inplace=True)

plant_summary['avg_water_ratio'] = plant_summary['avg_water_ratio'].replace([np.inf, -np.inf], np.nan)


In [527]:
plant_summary.head()

,plant_code,avg_water_ratio,suspicious_count
0,3,NaN,822
1,10,0.000000,182
2,26,0.078243,545
3,46,0.005579,0
4,47,0.000000,65


### 🔗 2. Merge with geo_df

In [529]:
geo_merged = geo_df.merge(plant_summary, left_on='plant_code', right_on='plant_code')

In [530]:
geo_merged[['Plant_Name', 'State', 'avg_water_ratio', 'suspicious_count']].head()


,Plant_Name,State,avg_water_ratio,suspicious_count
0,Barry,Alabama,NaN,822
1,Greene County,Alabama,0.000000,182
2,E C Gaston,Alabama,0.078243,545
3,Browns Ferry,Alabama,0.005579,0
4,Colbert,Alabama,0.000000,65


In [531]:
geo_merged.shape

(798, 35)

### 🌍 3. Set Up a Choropleth with folium
Plot avg_water_ratio or suspicious_count using circle markers:

In [628]:
from folium.plugins import MarkerCluster
# Centered on continental US
m = folium.Map(location=[39.8283, -98.5795], zoom_start=5, tiles='CartoDB positron')

# Optional: clustered markers
marker_cluster = MarkerCluster().add_to(m)

# Add points with correct color based on water ratio
for _, row in geo_merged.iterrows():
    popup_text = f"""
    <b>Plant:</b> {row['Plant_Name']}<br>
    <b>State:</b> {row['State']}<br>
    <b>Avg Water Ratio:</b> {row['avg_water_ratio']:.2f}<br>
    <b>Suspicious Count:</b> {int(row['suspicious_count'])}
    """
    
    # Logic for color
    if row['avg_water_ratio'] > 1:
        color = 'red'  # Suspicious (>1)
    elif row['avg_water_ratio'] == 0:
        color = 'yellow'  # Exactly 0
    else:
        color = 'green'  # Normal (0 < ratio <= 1)
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(marker_cluster)

# Add a legend for the map
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 150px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Water Ratio Legend</b><br>
     <i style="background:red; color:red; width:12px; height:12px; display:inline-block;"></i> Suspicious (>1)<br>
     <i style="background:yellow; color:yellow; width:12px; height:12px; display:inline-block;"></i> Zero (Water Ratio = 0)<br>
     <i style="background:green; color:green; width:12px; height:12px; display:inline-block;"></i> Normal (0 < ratio <= 1)<br>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Show the map in the notebook
m


## 🛠️ Map Logic
Circle colors: Based on whether the plant has any suspicious values (suspicious_count > 0).
- Red: Suspicious plants (water ratio > 1).
- Green: Normal plants (water ratio between 0 and 1, without issues).
- Yellow: Plants with water ratios exactly equal to 0 (which could be indicative of a reporting issue or a special case).

Popup content: Displays the Plant_Name, State, Avg Water Ratio, and Suspicious Count for each plant.

**Suspicious Water Ratio Map Interpretation:** 

Preliminary exploration of this map shows that plants with suspicious water usage ratios are geographically widespread, with no clear regional clustering. This suggests that geographic location alone is unlikely to explain anomalous water use. As a result, further analysis will focus on associations with cooling system configuration, fuel type, cooling technology, and generator characteristics. 

While this map only displays plants with suspicious positive water ratios, plants with suspicious ratios less than zero have also been flagged in the underlying data and will be included in future analyses.**

In [538]:
# Define your path and output file
path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus'
output_file = 'AZ_cooling_map_2.html'
full_path = os.path.join(path, output_file)

# Save the map
m.save(full_path)

# Confirm the export
if os.path.exists(full_path):
    print(f"🎉 Map saved successfully as '{output_file}' at:\n{full_path}")
else:
    print("❌ Error: Map was not saved.")


🎉 Map saved successfully as 'AZ_cooling_map_2.html' at:
/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus/AZ_cooling_map_2.html


In [540]:
mapped_df.shape

(677832, 77)

In [541]:
mapped_df['state'].head()

0    AL
1    AL
2    AL
3    AL
4    AL
Name: state, dtype: object

In [542]:
# Mapping of state abbreviations to full state names
abbreviation_to_fullname = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Convert state abbreviations in mapped_df to full names
mapped_df['State_full'] = mapped_df['state'].map(abbreviation_to_fullname)

In [543]:
mapped_df = mapped_df.drop(columns='state')  # if 'state' is abbreviated
mapped_df = mapped_df.rename(columns={'State_full': 'state'})

In [544]:
mapped_df['state'].head()

0    Alabama
1    Alabama
2    Alabama
3    Alabama
4    Alabama
Name: state, dtype: object

In [545]:
# Export data to csv
path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus'
output_file = 'cooling_boiler_GEO_trimmed.csv'
full_path = os.path.join(path, output_file)

# Save the dataframe
mapped_df.to_csv(full_path, index=False)

# Check if the file was created successfully
if os.path.exists(full_path):
    print(f"Saving confirmed: '{output_file}' has been created successfully.")
else:
    print("Error: File was not saved.")

Saving confirmed: 'cooling_boiler_GEO_trimmed.csv' has been created successfully.


In [546]:
state_missing = mapped_df.groupby('state').agg({
    'water_metadata_missing': 'mean',
    'water_metrics_missing': 'mean',
    'power_gen_data_missing': 'mean',
    'cooling_equipment_missing': 'mean'
}).reset_index()

state_missing['total_missing_percentage'] = state_missing[
    ['water_metadata_missing', 'water_metrics_missing', 'power_gen_data_missing', 'cooling_equipment_missing']
].mean(axis=1)


# 🗺️ 7b) Choropleth Map: Raw Counts of Missing Data v. Percentage Missing

In [590]:
import requests
import json

# U.S. States boundaries GeoJSON from a public source
geo_url = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'
states_geojson = requests.get(geo_url).json()

In [600]:
# Group by State and aggregate missing counts and coordinates
state_missing_2 = mapped_df.groupby('state').agg({
    'water_metadata_missing': 'sum',
    'water_metrics_missing': 'sum',
    'power_gen_data_missing': 'sum',
    'cooling_equipment_missing': 'sum',
    'Latitude': 'mean',
    'Longitude': 'mean'
}).reset_index()

# Sum total missing across all types
state_missing_2['total_missing_count'] = state_missing_2[
    ['water_metadata_missing', 'water_metrics_missing', 'power_gen_data_missing', 'cooling_equipment_missing']
].sum(axis=1)

# Get the total number of records per state
total_records_per_state = mapped_df.groupby('state').size().reset_index(name='total_records')

# Merge in total record counts
state_missing_2 = state_missing_2.merge(total_records_per_state, on='state')

# Calculate percentage of missing data (normalized)
state_missing_2['total_missing_percentage'] = state_missing_2['total_missing_count'] / (state_missing_2['total_records'] * 4)


In [624]:
import folium

# Create a new map object
choropleth_map_5 = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

# Add the Choropleth layer: base color as total_missing_count
folium.Choropleth(
    geo_data=geo_data,
    data=state_missing_2,
    columns=['state', 'total_missing_count'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',  # Base color intensity based on total missing data count
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Missing Data Count'
).add_to(choropleth_map_5)

# Add the overlay circles: size based on total_missing_percentage (normalized)
for idx, row in state_missing_2.iterrows():
    radius = row['total_missing_percentage'] * 20  # Adjust this scaling factor to control circle size

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=radius,  # Dynamic size based on total_missing_percentage
        color='blue',  # Color of the circle
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(choropleth_map_5)

# Show the map
choropleth_map_5


In [626]:
# Define your path and output file
path = '/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus'
output_file = 'AZ_choropleth_missingness_overlay.html'
full_path = os.path.join(path, output_file)

# Save the map
choropleth_map_5.save(full_path)

# Confirm the export
if os.path.exists(full_path):
    print(f"🎉 Map saved successfully as '{output_file}' at:\n{full_path}")
else:
    print("❌ Error: Map was not saved.")


🎉 Map saved successfully as 'AZ_choropleth_missingness_overlay.html' at:
/Users/amyzhang/Desktop/A6_Dashboard/AZ_sisyphus/AZ_choropleth_missingness_overlay.html


## 👀 Choropleth Map Analysis

The choropleth map with overlaid proportional symbols reveals that raw counts of missing data do not correlate directly with the rate of missingness across states. For example, states like **Texas, California, and Florida** exhibit some of the highest raw counts of missing data, yet their missingness rates are notably lower than those in states like **Alaska**. In fact, their rates are often comparable to or even lower than states with far fewer total records.

Conversely, many states with lower raw missing data counts also appear among those with the lowest relative missingness rates, reinforcing that raw volume and relative completeness are decoupled. This diverges from relationships observed in previous projects (e.g., total flu deaths vs. mortality rate), where raw counts more closely tracked normalized outcomes.

These findings suggest that if our predictive model aims to address state-level reporting of water usage metrics, then the **volume of reported data**—not just the rate of missingness—may need to be included as a feature. This is particularly relevant if reporting consistency varies with plant density, regulatory capacity, or other state-level attributes.